# forecasting selling price of chicken

Things to note:
* Changing how accuracy is being graded. As long as the model is able to predict the price within 20%, it is considered accurate. 
* Weekends are considered as a holiday as there would be many family gatherings and parties.

## importing the relevant libraries and dataset

In [1]:
# libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder

from sklearn import metrics
# from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import numpy as np
import xgboost as xgb

#!pip install category_encoders
import category_encoders as ce
from category_encoders import TargetEncoder


In [2]:
# reading file
df = pd.read_csv('/workspaces/forecasting/data/bigDataHoliday.csv')
df.head()

,UNIT,TAILS,KG,SALES PER KG,ABW,CHICKEN SIZE,PROVINCE,YEAR,MONTH,DAY,SupplyProvince,DemandProvince,Holiday
0,PAYAKUMBUH,1267,2856.0,18000.00000,2.254144,LARGE,SUMATERA BARAT,19,12,1,3885216,5993194,1
1,SINJAI,3310,7799.5,22637.89730,2.356344,LARGE,SULAWESI SELATAN,19,12,1,5447863,5931514,1
2,BANDAR JAYA,8236,22197.0,15905.01419,2.695119,LARGE,LAMPUNG,19,12,1,5404511,5618463,1
3,BANDAR LAMPUNG,2565,4971.0,18060.47073,1.938012,MEDIUM,LAMPUNG,19,12,1,5404511,5618463,1
4,BANYUASIN,696,1464.4,16368.88828,2.104023,LARGE,SUMATERA SELATAN,19,12,1,5758528,7021239,1


In [3]:
df.describe()

,TAILS,KG,SALES PER KG,ABW,YEAR,MONTH,DAY,SupplyProvince,DemandProvince,Holiday
count,15055.000000,15055.000000,15055.000000,15055.000000,15055.000000,15055.0,15055.000000,1.505500e+04,1.505500e+04,15055.000000
mean,8241.127532,15260.051938,19204.083254,1.829462,20.098439,12.0,15.780671,2.617008e+07,2.040207e+07,0.183726
std,7135.717864,13785.152507,2153.773552,0.336812,0.985753,0.0,8.879310,2.328217e+07,1.675878e+07,0.387274
min,15.000000,24.800000,6892.778929,0.547500,19.000000,12.0,1.000000,8.322200e+04,3.913950e+05,0.000000
25%,3235.500000,5698.000000,17414.197155,1.596241,19.000000,12.0,8.000000,4.424703e+06,5.141045e+06,0.000000
50%,6532.000000,11683.200000,19072.570490,1.821657,20.000000,12.0,16.000000,1.256251e+07,1.123831e+07,0.000000
75%,11104.000000,20707.450000,20576.154140,2.060000,21.000000,12.0,23.000000,4.994253e+07,3.453131e+07,0.000000
max,103280.000000,215666.400000,28621.861390,3.481132,23.000000,12.0,31.000000,6.499769e+07,5.193302e+07,1.000000


## train test split


In [4]:
# train test split before label encoding to prevent data leakage


X = df.drop('SALES PER KG', axis=1)
y = df['SALES PER KG']



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# checking for any nan values after splitting
print(X_train.isna().sum())
print(X_test.isna().sum())
print(y_train.isna().sum())
print(y_test.isna().sum())




UNIT              0
TAILS             0
KG                0
ABW               0
CHICKEN SIZE      0
PROVINCE          0
YEAR              0
MONTH             0
DAY               0
SupplyProvince    0
DemandProvince    0
Holiday           0
dtype: int64
UNIT              0
TAILS             0
KG                0
ABW               0
CHICKEN SIZE      0
PROVINCE          0
YEAR              0
MONTH             0
DAY               0
SupplyProvince    0
DemandProvince    0
Holiday           0
dtype: int64
0
0


In [5]:
# scaling numerical variable

scaler = StandardScaler()
columns_to_scale = ['KG', 'TAILS', 'SupplyProvince', 'DemandProvince']
X_train[columns_to_scale] = scaler.fit_transform(X_train[columns_to_scale])
X_test[columns_to_scale] = scaler.transform(X_test[columns_to_scale])

In [6]:
X_train

,UNIT,TAILS,KG,ABW,CHICKEN SIZE,PROVINCE,YEAR,MONTH,DAY,SupplyProvince,DemandProvince,Holiday
12179,SANGATTA,-0.868153,-0.903847,1.385429,SMALL,KALIMANTAN TIMUR,21,12,17,-1.024406,-0.964375,0
14448,SIJUNJUNG,-0.279093,-0.318778,1.739214,MEDIUM,SUMATERA BARAT,21,12,31,-0.995010,-0.935591,0
13769,BOJONEGORO,-0.943107,-0.881517,2.050574,LARGE,JAWA TIMUR,21,12,27,0.087111,0.845235,1
12070,BENGKULU,0.485797,0.670116,2.089701,LARGE,BENGKULU,21,12,16,-1.104780,-1.153716,0
11502,PALANGKARAYA,-0.470844,-0.300075,2.272398,LARGE,KALIMANTAN TENGAH,21,12,13,-1.060115,-1.051460,0
...,...,...,...,...,...,...,...,...,...,...,...,...
5191,GIANYAR,-0.105658,-0.134086,1.791001,MEDIUM,BALI,20,12,1,-0.918898,-0.925787,0
13418,50 KOTA,0.028610,-0.052511,1.721377,MEDIUM,SUMATERA BARAT,21,12,24,-0.995010,-0.935591,1
5390,CIREBON,0.012689,0.020821,1.864943,MEDIUM,JAWA BARAT,20,12,2,1.468594,1.720036,0
860,SUBANG(P),1.917237,1.183325,1.440357,SMALL,JAWA BARAT,19,12,5,1.665132,1.884218,0


# FUNCTIONS
- results(y_test, predictions):
- accuracy(y_test, predictions):

In [7]:
# defining new accuracy metrics
import math
def accuracy(y_test, predictions):
    cnt = 0
    length = len(predictions)

    for i in range(length):
        above = math.ceil(predictions[i] + (predictions[i] * 0.1)) # accuracy absed on 10% above and below
        below = math.ceil(predictions[i] - (predictions[i] * 0.1))

        if (predictions[i] == y_test[i]):
            cnt += 1
        elif (y_test[i] >= predictions[i] and y_test[i] <= above):
            cnt += 1
        elif (y_test[i] <= predictions[i] and y_test[i] >= below):
            cnt += 1
    actual = cnt / length
    return actual

In [8]:
# creating function to evaluate the model
def results(y_test, predictions):
  mae = mean_absolute_error(y_test, predictions)
  mse = mean_squared_error(y_test, predictions)
  rmse = np.sqrt(mse)
  r2 = r2_score(y_test, predictions)

  y_test_list = np.array(y_test).tolist()
  predictions_list = predictions.tolist()
  acc = accuracy(y_test_list, predictions_list)

  print(f"Mean Absolute Error (MAE): {mae}")
  print(f"Mean Squared Error (MSE): {mse}")
  print(f"Root Mean Squared Error (RMSE): {rmse}")
  print(f"R-squared (R²): {r2}")
  print(f"Re-Defined Accuracy: {acc}")

# encoding

In [9]:
# encoding of columns
# label encoding for province

label_encoder = LabelEncoder()
X_train['SIZE ENCODE'] = label_encoder.fit_transform(X_train['CHICKEN SIZE'])
X_test['SIZE ENCODE'] = label_encoder.transform(X_test['CHICKEN SIZE'])

# dropping chicken size column
X_train = X_train.drop(['CHICKEN SIZE'], axis = 1)
X_test = X_test.drop(['CHICKEN SIZE'], axis = 1)


# one hot encoding for province

encoder = OneHotEncoder(handle_unknown='ignore')
encoder.fit(X_train[['PROVINCE']])
X_train_encoded = encoder.transform(X_train[['PROVINCE']])
X_test_encoded = encoder.transform(X_test[['PROVINCE']])

# Convert the encoded matrices back to dataframes for easier manipulation
columns = encoder.get_feature_names_out(['PROVINCE'])
X_train_encoded_df = pd.DataFrame(X_train_encoded.toarray(), columns=columns, index=X_train.index)
X_test_encoded_df = pd.DataFrame(X_test_encoded.toarray(), columns=columns, index=X_test.index)

# merge back the dataframe
X_train = pd.concat([X_train.drop('PROVINCE', axis=1), X_train_encoded_df], axis=1)
X_test = pd.concat([X_test.drop('PROVINCE', axis=1), X_test_encoded_df], axis=1)


# one hot encoding for unit
encoder = OneHotEncoder(handle_unknown='ignore')
encoder.fit(X_train[['UNIT']])
X_train_encoded_unit = encoder.transform(X_train[['UNIT']])
X_test_encoded_unit = encoder.transform(X_test[['UNIT']])

# Convert the encoded matrices back to dataframes for easier manipulation
columns = encoder.get_feature_names_out(['UNIT'])
X_train_encoded_df_unit = pd.DataFrame(X_train_encoded_unit.toarray(), columns=columns, index=X_train.index)
X_test_encoded_df_unit = pd.DataFrame(X_test_encoded_unit.toarray(), columns=columns, index=X_test.index)

# merge back the dataframe, dropping unit column
X_train = pd.concat([X_train.drop('UNIT', axis=1), X_train_encoded_df_unit], axis=1)
X_test = pd.concat([X_test.drop('UNIT', axis=1), X_test_encoded_df_unit], axis=1)




# Linear Regression

In [18]:
#Applying Logistic Regression
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)

# predictions
predictions_lr = lr.predict(X_test)

# print metrics
print(results(y_test, predictions_lr))



'''# Use the custom accuracy function
accuracy_score = accuracy(y_test_list, predictions_list)
print(f"Custom Accuracy Score: {accuracy_score}")'''

Mean Absolute Error (MAE): 1096.9887937565593
Mean Squared Error (MSE): 2204010.1056202706
Root Mean Squared Error (RMSE): 1484.5908882989518
R-squared (R²): 0.5373888190756129
Re-Defined Accuracy: 0.8465626037861176
None


'# Use the custom accuracy function\naccuracy_score = accuracy(y_test_list, predictions_list)\nprint(f"Custom Accuracy Score: {accuracy_score}")'

# Random Forest



In [19]:
# random forest model
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = 400) # 400 trees
rf.fit(X_train, y_train)

# predictions
predictions_rf = rf.predict(X_test)

# print accuracy score
print(results(y_test, predictions_rf))

Mean Absolute Error (MAE): 481.8099547746834
Mean Squared Error (MSE): 690170.1718080787
Root Mean Squared Error (RMSE): 830.7648113684635
R-squared (R²): 0.855136581540733
Re-Defined Accuracy: 0.9637994021919628
None


# Gradient Boosting

## XGBoost

In [20]:
# applying xgboost
xg = xgb.XGBRegressor()

# fitting the model
xg.fit(X_train, y_train)

# predictions
predictions_xg = xg.predict(X_test) 

# print accuracy score
print(results(y_test, predictions_xg))

Mean Absolute Error (MAE): 603.0214842705082
Mean Squared Error (MSE): 853160.8484241929
Root Mean Squared Error (RMSE): 923.6670657895045
R-squared (R²): 0.8209256179898987
Re-Defined Accuracy: 0.9554965127864496
None


# Decision Tree

In [12]:
# applying decision trees
from sklearn.tree import DecisionTreeRegressor
dt = DecisionTreeRegressor()
dt.fit(X_train, y_train)

# predictions
predictions_dt = dt.predict(X_test)
print(results(y_test, predictions_dt))

Mean Absolute Error (MAE): 622.3767745532588
Mean Squared Error (MSE): 1301568.8692961177
Root Mean Squared Error (RMSE): 1140.8632123511204
R-squared (R²): 0.7268069188321427
Re-Defined Accuracy: 0.9302557289936898
None


# SVM

In [14]:
# applying SVM
from sklearn.svm import SVR
svm = SVR()
svm.fit(X_train, y_train)

# predictions
predictions_SVM = svm.predict(X_test)
print(results(y_test, predictions_SVM))

Mean Absolute Error (MAE): 1714.9721868000245
Mean Squared Error (MSE): 4621680.866075736
Root Mean Squared Error (RMSE): 2149.8094952985334
R-squared (R²): 0.029931288491422237
Re-Defined Accuracy: 0.6021255396878114
None


# Extra Trees Regressor


In [10]:
# applying extra trees regressor
from sklearn.ensemble import ExtraTreesRegressor
et = ExtraTreesRegressor()
et.fit(X_train, y_train)

# predictions
predictions_et = et.predict(X_test)
print(results(y_test, predictions_et))

Mean Absolute Error (MAE): 453.8822808106234
Mean Squared Error (MSE): 658191.602126477
Root Mean Squared Error (RMSE): 811.2900850660491
R-squared (R²): 0.8618487303856746
Re-Defined Accuracy: 0.9684490202590501
None


# Neural Network



In [17]:
# applying neural network
#!pip install tensorflow

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import layers


# creating neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)  # Single output node for regression
])

# compiling the model
model.compile(optimizer='adam', loss='mean_squared_error')

# training the model
model.fit(X_train, y_train, epochs = 100, batch_size = 32)


# print accuracy score
def accuracy_nn(y_test, predictions): # new accuracy metrics for neural networks
    cnt = 0
    length = len(predictions)

    for i in range(length):
        prediction = predictions[i][0] if isinstance(predictions[i], list) else predictions[i]
        above = math.ceil(prediction + (prediction * 0.1))
        below = math.ceil(prediction - (prediction * 0.1))

        if prediction == y_test[i]:
            cnt += 1
        elif y_test[i] >= prediction and y_test[i] <= above:
            cnt += 1
        elif y_test[i] <= prediction and y_test[i] >= below:
            cnt += 1

    actual = cnt / length
    return actual


# predictions
predictions_nn = model.predict(X_test)
predictions_nn = predictions_nn.flatten()

print(results(y_test, predictions_nn))


Epoch 1/100
377/377 [==============================] - 1s 1ms/step - loss: 248296928.0000
Epoch 2/100
377/377 [==============================] - 0s 1ms/step - loss: 11056535.0000
Epoch 3/100
377/377 [==============================] - 0s 1ms/step - loss: 7033267.5000
Epoch 4/100
377/377 [==============================] - 0s 1ms/step - loss: 4480553.5000
Epoch 5/100
377/377 [==============================] - 0s 1ms/step - loss: 3256021.5000
Epoch 6/100
377/377 [==============================] - 0s 1ms/step - loss: 2868693.7500
Epoch 7/100
377/377 [==============================] - 0s 1ms/step - loss: 2731434.2500
Epoch 8/100
377/377 [==============================] - 0s 987us/step - loss: 2645546.0000
Epoch 9/100
377/377 [==============================] - 0s 1ms/step - loss: 2577673.2500
Epoch 10/100
377/377 [==============================] - 0s 1ms/step - loss: 2521572.7500
Epoch 11/100
377/377 [==============================] - 0s 1ms/step - loss: 2472130.0000
Epoch 12/100
377/377 [===